In [1]:
!pwd

/Users/abiolatresordjigui/DM/streamlit-apps


In [2]:
!pip3 install polars

In [3]:
!pip3 install pandas

In [4]:
import polars as pl

In [5]:
!pip install faiss-cpu

In [6]:
import faiss

In [7]:
def build_faiss_index(embeddings):
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    faiss.normalize_L2(embeddings.cpu().numpy())
    index.add(embeddings.cpu().numpy())
    return index

In [8]:
extracted_keywords = pl.read_parquet("/Users/abiolatresordjigui/DM/streamlit-apps/colagte_freq_df.parquet")

In [29]:
extracted_keywords_filtered = extracted_keywords#.filter(pl.col("nb_of_docs")>=4)

In [30]:
extracted_keywords_filtered.shape

(1004609, 6)

In [11]:
!pip3 install sentence-transformers

In [12]:
from sentence_transformers import SentenceTransformer, util

/opt/anaconda3/envs/keyword_suggestor/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/opt/anaconda3/envs/keyword_suggestor/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
extracted_list_of_keywords = list(set(extracted_keywords_filtered.select(pl.col("lemma")).to_series().to_list()))
print("Total number of keywords: ", len(extracted_list_of_keywords))

Total number of keywords:  1004609


In [15]:
import pandas as pd
pd.DataFrame({"Keyword": extracted_list_of_keywords}).to_pickle('/Users/abiolatresordjigui/DM/streamlit-apps/extracted_list_of_keywords.pkl')

In [16]:
# documents = [
#     "The recent political reforms have sparked debates over education funding, with many advocating for increased investment in schools to bridge the learning gap between urban and rural areas.",
#     "Football has become a powerful tool for political leaders to rally support, as they often attend high-profile matches to connect with voters and demonstrate national pride.",
#     "A key issue in the upcoming elections is whether to prioritize public education reform, with candidates proposing diverse strategies to improve teacher training and student outcomes.",
#     "Political figures are increasingly using football events as platforms for promoting unity, with major tournaments providing opportunities to soften public opinion on divisive policies.",
#     "The government’s new initiative to expand access to higher education has sparked controversy, as some critics argue that it will strain an already overburdened public school system."
# ]

In [17]:
import torch
use_cuda = torch.cuda.is_available()

In [18]:
use_cuda

False

In [21]:
corpus_embeddings = model.encode(extracted_list_of_keywords, convert_to_tensor=True)

In [22]:
corpus_embeddings.cpu().numpy()[0].shape

(384,)

In [23]:
index = build_faiss_index(corpus_embeddings)

In [24]:
faiss.write_index(index, "/Users/abiolatresordjigui/DM/streamlit-apps/extracted_keywords_index.bin")

In [25]:
index2 = faiss.read_index("/Users/abiolatresordjigui/DM/streamlit-apps/extracted_keywords_index.bin")

In [26]:
import numpy as np
np.save('/Users/abiolatresordjigui/DM/streamlit-apps/extracted_keywords_embeddings.npy', corpus_embeddings.cpu())

In [27]:
type(index)

faiss.swigfaiss.IndexFlatIP

In [28]:
help(index.search)

Help on method replacement_search in module faiss.class_wrappers:

replacement_search(x, k, *, params=None, D=None, I=None) method of faiss.swigfaiss.IndexFlatIP instance
    Find the k nearest neighbors of the set of vectors x in the index.

    Parameters
    ----------
    x : array_like
        Query vectors, shape (n, d) where d is appropriate for the index.
        `dtype` must be float32.
    k : int
        Number of nearest neighbors.
    params : SearchParameters
        Search parameters of the current search (overrides the class-level params)
    D : array_like, optional
        Distance array to store the result.
    I : array_like, optional
        Labels array to store the results.

    Returns
    -------
    D : array_like
        Distances of the nearest neighbors, shape (n, k). When not enough results are found
        the label is set to +Inf or -Inf.
    I : array_like
        Labels of the nearest neighbors, shape (n, k).
        When not enough results are found,

In [2]:
!pip3 install cloud-storage

  Using cached protobuf-5.28.2-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 8.1 MB/s eta 0:00:00a 0:00:01
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached protobuf-5.28.2-cp38-abi3-macosx_10_9_universal2.whl (414 kB)
